In [23]:
pip install requests

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: requests in c:\users\vian martinez\appdata\local\programs\python\python312\lib\site-packages (2.32.3)




[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [24]:
pip install torch

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [25]:
pip install transformers

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [26]:
import requests
import json
import torch
from transformers import XLNetTokenizer, XLNetForSequenceClassification

In [27]:
GET_ONM_DATA_API_URL = "http://174.138.31.241:6178/online-media/compute/getArticle"


In [18]:
# Load the trained sentiment analysis model and tokenizer
MODEL_DIR = "./xlnet_sentiment_model_v2"
TOKENIZER_DIR = "./xlnet_sentiment_tokenizer_v2"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [19]:
# Load the saved model and tokenizer
model = XLNetForSequenceClassification.from_pretrained(MODEL_DIR).to(device)
tokenizer = XLNetTokenizer.from_pretrained(TOKENIZER_DIR)

OSError: Incorrect path_or_model_id: './xlnet_sentiment_model_v2'. Please provide either the path to a local folder or the repo_id of a model on the Hub.

In [4]:
# Map label indices to sentiments
label_map = {0: "negative", 1: "neutral", 2: "positive"}

In [5]:
def get_user_input():
    print("Enter the following details to fetch articles from the API:")
    project_id = input("Project ID: ")
    clipping_id = input("Clipping ID: ")
    end_date = input("End Date (YYYY-MM-DD HH:MM:SS): ")
    start_date = input("Start Date (YYYY-MM-DD HH:MM:SS): ")
    page = input("Page (default 1): ") or "1"
    limit = input("Limit (default 10): ") or "10"
    validation_list = input("Validation List (e.g., unvalidated): ")

    # Construct the request body
    body = {
        "projectId": project_id,
        "clippingId": clipping_id,
        "endDate": end_date,
        "startDate": start_date,
        "page": int(page),
        "limit": int(limit),
        "validationList": [validation_list]
    }

    return body


In [6]:
def analyze_sentiment(text):
    """Predict sentiment of a given text using the trained model."""
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    inputs = {key: val.to(device) for key, val in inputs.items()}
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    predicted_label = torch.argmax(logits, dim=1).item()
    return label_map[predicted_label]

In [7]:
def fetch_articles(api_url, body):
    headers = {
        "Content-Type": "application/json"
    }
    print("\nSending request to the API...")
    print("Request Body:", json.dumps(body, indent=4))  # Debugging request body
    try:
        response = requests.post(api_url, json=body, headers=headers)
        if response.status_code == 200:
            articles = response.json()
            print("\nArticles fetched successfully!")
            
            for article in articles:
                print("\nAnalyzing sentiment for article:")
                print(f"Title: {article['title']}")
                print(f"Original Sentiment: {article.get('sentiment', 'neutral')}")

                # Analyze and update sentiment
                new_sentiment = analyze_sentiment(article['content'])
                article['sentiment'] = new_sentiment

                print(f"Updated Sentiment: {new_sentiment}")
            
            print("\nUpdated Articles with New Sentiments:")
            print(json.dumps(articles, indent=4))
        else:
            print(f"\nFailed to fetch articles. Status Code: {response.status_code}")
            print(f"Error: {response.text}")
    except Exception as e:
        print(f"\nAn error occurred: {e}")

In [9]:

if __name__ == "__main__":
    request_body = get_user_input()
    fetch_articles(GET_ONM_DATA_API_URL, request_body)

Enter the following details to fetch articles from the API:

Sending request to the API...
Request Body: {
    "projectId": "c960b616-7847-472f-9e16-e47941536fd7",
    "clippingId": "00aa0060-4295-499c-b403-6d1bea15a020",
    "endDate": "2024-12-15 23:59:59",
    "startDate": "2024-12-15 00:00:00",
    "page": 1,
    "limit": 10,
    "validationList": [
        "unvalidated"
    ]
}

Articles fetched successfully!

Analyzing sentiment for article:
Title: Nokia 6600 5G Ultra: Hadir dengan Desain Klasik, Layar AMOLED 6,2inci dan Chipset Snapdragon 888
Original Sentiment: neutral

An error occurred: name 'tokenizer' is not defined
